In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2024-06-17 05:49:36.608107: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 05:49:36.608229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 05:49:36.730518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available:  1


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet152V2

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from PIL import Image
from sklearn.metrics import classification_report,confusion_matrix

In [3]:
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor

def create_directory_structure(base_dir, class_names):
    for class_name in class_names:
        os.makedirs(os.path.join(base_dir, class_name), exist_ok=True)

def copy_file(file_path, destination_folder):
    class_name = os.path.basename(os.path.dirname(file_path))
    shutil.copy(file_path, os.path.join(destination_folder, class_name))

def copy_files(file_list, destination_folder):
    with ThreadPoolExecutor() as executor:
        for file_path in file_list:
            executor.submit(copy_file, file_path, destination_folder)

# Set paths
input_folder = "/kaggle/input/leaf-dataset-main/Leaf_dataset_main"
output_folder = "Tr-Te-Val"
train_folder = os.path.join(output_folder, 'train')
val_folder = os.path.join(output_folder, 'val')
test_folder = os.path.join(output_folder, 'test')

# List all class folders
class_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]

# Create output directories
create_directory_structure(train_folder, class_folders)
create_directory_structure(val_folder, class_folders)
create_directory_structure(test_folder, class_folders)

# Process each class separately
for class_folder in class_folders:
    class_path = os.path.join(input_folder, class_folder)
    files = [os.path.join(class_path, f) for f in os.listdir(class_path) if f.endswith(('jpg', 'jpeg', 'png'))]
    
    # Limit to at most 100 images per class
    files = files[:100]
    total_files = len(files)
    
    # Calculate split sizes
    train_size = int(total_files * 0.8)
    val_size = int(total_files * 0.2)
    test_size = int(total_files * 0.15)

    # Shuffle files to ensure random distribution
    random.shuffle(files)

    # Split into train, validation, and test
    train_files = files[:train_size]
    val_files = files[train_size:train_size + val_size]
    test_files = files[:test_size]
    
    # Copy files to respective folders
    copy_files(train_files, train_folder)
    copy_files(val_files, val_folder)
    copy_files(test_files, test_folder)

print("Dataset splitting complete!")


Dataset splitting complete!


In [4]:
# Create ImageDataGenerator with augmentation for train and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

# Create train generator
train_generator = datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Create validation generator (no need for subset='validation')
val_generator = datagen.flow_from_directory(
    val_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Create test generator (no augmentation, no validation split)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Do not shuffle test data
)

Found 10810 images belonging to 170 classes.
Found 2680 images belonging to 170 classes.
Found 2003 images belonging to 170 classes.


In [ ]:
a = train_generator.class_indices
classes = list(a.keys())
print(classes)
b = test_generator.class_indices
classes = list(b.keys())
print(classes)
c = val_generator.class_indices
classes = list(c.keys())
print(classes)

In [5]:
# Convert to tf.data.Dataset for better performance
num_classes = 170

In [6]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

# Define the path to the locally downloaded weights file
weights_path = '/kaggle/input/resnet50v2/tensorflow2/resnet50v2/1/resnet50v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Load the ResNet152V2 model with the local weights file
base_model = ResNet50V2(include_top=False,
                         input_shape=(224, 224, 3),
                         weights=weights_path,
                         pooling='avg')


In [7]:
# import tensorflow as tf
# from tensorflow.keras.applications.resnet_v2 import ResNet152V2

# # Define the path to the locally downloaded weights file
# weights_path = '/kaggle/input/resnet152v2/tensorflow2/resnet152v2_weights_tf_dim_ordering_tf_kernels_notop/1/resnet152v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

# # Load the ResNet152V2 model with the local weights file
# base_model = ResNet152V2(
#     include_top=False,
#     input_shape=(224, 224, 3),
#     weights=weights_path,  # Load weights manually
#     pooling='avg'
# )

In [8]:
# from tensorflow.keras.layers import Reshape

# # Ensure the output shape is compatible
# x = base_model.output
# print(f'Base model output shape: {x.shape}')  # Should be a 4D tensor

# # Reshape the output to prepare for GlobalAveragePooling2D
# x = Reshape((1, 1, 2048))(x)
# print(f'Base model output shape: {x.shape}')

In [9]:
x = base_model.output
x = BatchNormalization()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [10]:
# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

initial_learning_rate = 0.001
model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [12]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [14]:
# Train the model
hist = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 668s 2s/step - accuracy: 0.6189 - loss: 2.2104 - val_accuracy: 0.7627 - val_loss: 1.7515 - learning_rate: 0.0010
Epoch 2/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 645s 2s/step - accuracy: 0.6337 - loss: 2.1437 - val_accuracy: 0.7709 - val_loss: 1.7055 - learning_rate: 0.0010
Epoch 3/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 652s 2s/step - accuracy: 0.6492 - loss: 2.0708 - val_accuracy: 0.7705 - val_loss: 1.6868 - learning_rate: 0.0010
Epoch 4/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 644s 2s/step - accuracy: 0.6517 - loss: 2.0319 - val_accuracy: 0.7780 - val_loss: 1.6694 - learning_rate: 0.0010
Epoch 5/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 644s 2s/step - accuracy: 0.6571 - loss: 2.0350 - val_accuracy: 0.7746 - val_loss: 1.6311 - learning_rate: 0.0010
Epoch 6/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - accuracy: 0.6663 - loss: 1.9982 - val_accuracy: 0.7724 - val_loss: 1.6477 - learning_rate: 0.0010
Epoch 7/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 641s 2s/step - accuracy: 0.6754 - loss: 1.

In [16]:
model.save('17062410epochs.keras')

In [17]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [18]:
fine_tuning_learning_rate = 1e-4
model.compile(optimizer=Adam(learning_rate=fine_tuning_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Continue training
hist1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/10
290/338 ━━━━━━━━━━━━━━━━━━━━ 1:15 2s/step - accuracy: 0.6382 - loss: 2.0613

In [ ]:
model.save('17062410epochs.keras')

In [ ]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [ ]:
# Continue training
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
)

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}')

# Save the model in .keras format
model.save('leaf_classification_model.keras')


In [ ]:
model = load_model("./Model.h5")

In [ ]:
#test
model.evaluate(test_ds)

**Graphs to show loss and accuracy along the training**

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(hist.epoch,hist.history['accuracy'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.epoch,hist.history['loss'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()

**Making Predictions and Verifying**

In [ ]:
preds = model.predict(test_ds,verbose=1)

In [ ]:
pred = [np.argmax(i) for i in preds]
pred[:5]

In [ ]:
actual = test_ds.classes

**Classification Report**

In [ ]:
print(classification_report(pred,actual))

**Confusion Matrix**

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(pred,actual),annot = True, fmt = 'd', cmap = 'Blues');

**Individual image Identification**

> function to predict

In [ ]:
def predict_img(path):
    img = image.load_img(path,target_size=(224,224))
    img_arr = image.img_to_array(img)
    img_arr_expnd  = np.expand_dims(img_arr,axis=0)
    img = keras.applications.resnet_v2.preprocess_input(img_arr_expnd)

    pred = model.predict(img)
    result = classes[np.argmax(pred)]

    return result

In [ ]:
plt.imshow(Image.open("../input/flowers-recognition/flowers/rose/10503217854_e66a804309.jpg"))
plt.axis('off');

**Prediction**

In [ ]:
print("The given image belongs to class:",predict_img("../input/flowers-recognition/flowers/rose/10503217854_e66a804309.jpg"))